In [1]:
import os
import torch
import numpy as np
import mmcv
import json
import matplotlib.pyplot as plt
import itertools
from mmengine.config import Config
from mmengine.runner import Runner
from mmengine.registry import DATASETS
from mmdet.utils import register_all_modules
from mmdet.apis import inference_detector, init_detector
from mmdet.visualization import DetLocalVisualizer

# ✅ Paksa torch.load agar selalu pakai weights_only=False
# Ini penting untuk kompatibilitas PyTorch versi baru
_orig_torch_load = torch.load
def torch_load_wrapper(*args, **kwargs):
    kwargs["weights_only"] = False
    return _orig_torch_load(*args, **kwargs)

torch.load = torch_load_wrapper

# ========== KONFIG DASAR ==========
DEBUG_MODE = False
print("="*50)
print(f"      MODE DEBUGGING: {'AKTIF' if DEBUG_MODE else 'NONAKTIF'}")
print("="*50)

register_all_modules()

print("--- Tes Ketersediaan GPU PyTorch ---")
print(f"Apakah CUDA tersedia? -> {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU terdeteksi: {torch.cuda.get_device_name(0)}")
else:
    print("PERINGATAN: Tidak ada GPU. Training di CPU akan lambat.")

      MODE DEBUGGING: NONAKTIF
--- Tes Ketersediaan GPU PyTorch ---
Apakah CUDA tersedia? -> True
GPU terdeteksi: NVIDIA GeForce RTX 5060


In [2]:
# ===== Step 1: Load Config RetinaNet + Swin Backbone =====
print("--- INFO: Memilih model backbone Swin Transformer ---")
cfg_path = 'mmdetection/configs/swin/retinanet_swin-s-p4-w7_fpn_1x_coco_custom.py'
cfg = Config.fromfile(cfg_path)

model_name = "retinanet" if "retinanet" in cfg_path.lower() else "custommodel"
print(f"--- INFO: Model yang digunakan: {model_name} ---")

--- INFO: Memilih model backbone Swin Transformer ---
--- INFO: Model yang digunakan: retinanet ---


In [3]:
# ===== Step 2: Definisikan Pipeline Dasar =====
print("--- INFO: Mendefinisikan pipeline dasar untuk training dan validasi ---")

train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
    dict(type='Resize', scale=(416, 416), keep_ratio=True),
    dict(type='Pad', size=(416, 416), pad_val=dict(img=(114, 114, 114))),
]

val_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=(416, 416), keep_ratio=True),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False),
    dict(type='PackDetInputs',
         meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape', 'scale_factor'))
]

print("--- INFO: Definisi pipeline selesai. ---")


--- INFO: Mendefinisikan pipeline dasar untuk training dan validasi ---
--- INFO: Definisi pipeline selesai. ---


In [4]:
# ===== Step 3: Dataset & Kelas =====
CLASSES = (
    'Betta', 'Discuss', 'Glofish', 'Goldfish', 'Guppy', 'Gurami',
    'Manfish', 'Molly', 'Swordtail', 'Tiger Barb'
)
cat_id_map = {
    1: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 9: 6, 10: 7, 11: 8, 12: 9
}
PALETTE = [
    (220, 20, 60), (0, 128, 255), (60, 179, 113), (255, 140, 0), (138, 43, 226),
    (255, 215, 0), (199, 21, 133), (127, 255, 212), (70, 130, 180), (255, 99, 71)
]
metainfo = {'classes': CLASSES, 'palette': PALETTE, 'cat2label': cat_id_map}

print(f"--- INFO: Konfigurasi diatur untuk melatih {len(CLASSES)} kelas. ---")

data_root = 'dataset/10_jenis_ikan/'
train_ann_file = 'train/_annotations.coco.json'
val_ann_file   = 'valid/_annotations.coco.json'

# --- Konfigurasi Dataset ---
train_dataset_cfg = dict(
    type='CocoDataset',
    data_root=data_root,
    metainfo=metainfo,
    ann_file=train_ann_file,
    data_prefix=dict(img='train/'),
    filter_cfg=dict(filter_empty_gt=True),
    pipeline=train_pipeline
)
cfg.train_dataloader.dataset = dict(
    type='MultiImageMixDataset',
    dataset=train_dataset_cfg,
    pipeline=[
        dict(type='PhotoMetricDistortion'),
        dict(type='RandomFlip', prob=0.5),
        dict(type='Normalize',
             mean=[123.675, 116.28, 103.53],
             std=[58.395, 57.12, 57.375],
             to_rgb=True),
        dict(type='PackDetInputs')
    ]
)
cfg.val_dataloader.dataset.update(dict(
    data_root=data_root, metainfo=metainfo, ann_file=val_ann_file,
    data_prefix=dict(img='valid/'), pipeline=val_pipeline
))
cfg.test_dataloader.dataset.update(dict(
    data_root=data_root, metainfo=metainfo, ann_file=val_ann_file,
    data_prefix=dict(img='valid/'), pipeline=val_pipeline
))

cfg.train_dataloader.batch_size = 8
cfg.val_dataloader.batch_size = 8
cfg.test_dataloader.batch_size = 8
cfg.train_dataloader.num_workers = 4
cfg.val_dataloader.num_workers = 4
cfg.test_dataloader.num_workers = 4
print(f"--- INFO: Batch size: {cfg.train_dataloader.batch_size}, Num workers: {cfg.train_dataloader.num_workers} ---")

--- INFO: Konfigurasi diatur untuk melatih 10 kelas. ---
--- INFO: Batch size: 8, Num workers: 4 ---


In [5]:
# ===== Step 4: Evaluator COCO (bbox) + classwise =====
cfg.val_evaluator.ann_file = os.path.join(data_root, val_ann_file)
cfg.test_evaluator.ann_file = os.path.join(data_root, val_ann_file)
cfg.val_evaluator.metric = 'bbox'
cfg.test_evaluator.metric = 'bbox'
cfg.val_evaluator.classwise = True
cfg.test_evaluator.classwise = True
cfg.val_evaluator.outfile_prefix = os.path.join('work_eval', 'val_results')
cfg.test_evaluator.outfile_prefix = os.path.join('work_eval', 'test_results')
os.makedirs('work_eval', exist_ok=True)

In [6]:
# ===== Step 5: Atur jumlah kelas di head =====
if hasattr(cfg.model, 'bbox_head'):
    cfg.model.bbox_head.num_classes = len(CLASSES)
elif hasattr(cfg.model, 'roi_head'):
    if isinstance(cfg.model.roi_head.bbox_head, list):
        for head in cfg.model.roi_head.bbox_head:
            head.num_classes = len(CLASSES)
    else:
        cfg.model.roi_head.bbox_head.num_classes = len(CLASSES)

In [7]:
# ===== Step 6: Training Settings =====
cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=0.0001, weight_decay=0.05),
    paramwise_cfg=dict(custom_keys={'norm': dict(decay_mult=0.)})
)
cfg.param_scheduler = []
print("--- INFO: Menggunakan Optimizer AdamW dengan LR statis 0.00001 ---")

best_checkpoint_path = './outputs_bs8_lr0001_workers4/epoch_1488.pth'
if not DEBUG_MODE and os.path.exists(best_checkpoint_path):
    cfg.load_from = best_checkpoint_path
    print(f"--- INFO: Memuat bobot dari: {cfg.load_from} ---")
else:
    cfg.load_from = None
    print("--- INFO: Memulai training dari awal (scratch). ---")

cfg.train_cfg.max_epochs = 30 if not DEBUG_MODE else 2
cfg.default_hooks.logger.interval = 10
cfg.default_hooks.checkpoint.interval = 1
cfg.default_hooks.checkpoint.max_keep_ckpts = 3
cfg.default_hooks.checkpoint.save_best = 'coco/bbox_mAP'
cfg.default_hooks.checkpoint.rule = 'greater'
cfg.visualizer.vis_backends = [
    dict(type='LocalVisBackend'),
    dict(type='TensorboardVisBackend')
]
print(f"--- INFO: Training akan berjalan selama {cfg.train_cfg.max_epochs} epoch. ---")

--- INFO: Menggunakan Optimizer AdamW dengan LR statis 0.00001 ---
--- INFO: Memulai training dari awal (scratch). ---
--- INFO: Training akan berjalan selama 30 epoch. ---


In [8]:
# ===== Step 7: Work dir =====
work_dir_base = f'./outputs_model_dengan_grafik'
cfg.work_dir = f"{work_dir_base}_debug" if DEBUG_MODE else work_dir_base
print(f"--- INFO: Output ke: {cfg.work_dir} ---")
os.makedirs(cfg.work_dir, exist_ok=True)

--- INFO: Output ke: ./outputs_model_dengan_grafik ---


In [9]:
# ===== Step 8: Membangun Dataset untuk Pengecekan =====
print("\n--- Membangun Dataset ---")
try:
    train_dataset = DATASETS.build(cfg.train_dataloader.dataset)
    val_dataset = DATASETS.build(cfg.val_dataloader.dataset)
    print(f"Jumlah data training: {len(train_dataset)}")
    print(f"Jumlah data validasi: {len(val_dataset)}")
except Exception as e:
    print(f"\nFATAL ERROR memuat dataset: {e}")
    raise e
print("---------------------------------\n")

print("Konfigurasi selesai. Siap training.")


--- Membangun Dataset ---
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Jumlah data training: 1690
Jumlah data validasi: 484
---------------------------------

Konfigurasi selesai. Siap training.


In [10]:
# ===== Step 9: Training =====
print(">>> Mulai training...")
runner = Runner.from_cfg(cfg)
runner.train()
print(">>> Training selesai.")


>>> Mulai training...
10/29 10:10:47 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 699190573
    GPU 0: NVIDIA GeForce RTX 5060
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
    NVCC: Cuda compilation tools, release 12.8, V12.8.61
    MSVC: n/a, reason: fileno
    PyTorch: 2.8.0+cu128
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 201703
  - MSVC 193833145
  - Intel(R) oneAPI Math Kernel Library Version 2025.2-Product Build 20250620 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.7.1 (Git Hash 8d263e693366ef8db40acc569cc7d8edf644556d)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 12.8
  - NVCC archi

In [11]:
# =================================================================================
# BAGIAN BARU: ANALISIS DAN PLOTTING OTOMATIS SETELAH TRAINING
# =================================================================================

# --- Direktori Laporan ---
report_dir = os.path.join(cfg.work_dir, 'reports_epoch30_batchsize8_lr0.0001(2)')
os.makedirs(report_dir, exist_ok=True)
print(f"\n--- Memulai Analisis Pasca-Training. Laporan akan disimpan di: {report_dir} ---")


# ========== LANGKAH 1: EVALUASI STANDAR MMDETECTION (UNTUK coco_eval_summary.txt) ==========
print("\n>>> Menjalankan evaluasi COCO standar...")
eval_results = runner.test() 

# Simpan ringkasan evaluasi standar ke file teks
eval_summary_path = os.path.join(report_dir, 'coco_eval_summary.txt')
with open(eval_summary_path, 'w', encoding='utf-8') as f:
    f.write("=== COCO BBox Evaluation (mAP) ===\n")
    for k, v in eval_results.items():
        # Filter untuk hanya menulis metrik yang relevan (mAP dan presisi per kelas)
        if 'mAP' in k or 'precision' in k:
            f.write(f"{k}: {v:.4f}\n")
print(f">>> Ringkasan evaluasi COCO disimpan di: {eval_summary_path}")

# ========== FUNGSI UNTUK PLOTTING KURVA TRAINING ==========
# ========== FUNGSI UNTUK PLOTTING KURVA TRAINING (VERSI FINAL YANG BENAR) ==========
def create_stability_curve_by_epoch(work_dir, output_dir):
    print("\n>>> Membuat grafik stabilitas training (Loss vs. mAP)...")
    
    # Bagian pencarian file ini sudah benar dan akan bekerja secara otomatis
    log_file = './outputs_model_dengan_grafik_epoch300_batchsize4_lr0.00001/20251028_170807/vis_data/20251028_170807.json'
    vis_data_path = os.path.join(work_dir, 'vis_data')
    if os.path.exists(vis_data_path):
        # Cari file .json, tapi abaikan scalars.json
        log_files = [f for f in os.listdir(vis_data_path) if f.endswith('.json') and 'scalars' not in f]
        if log_files:
            # Urutkan berdasarkan waktu modifikasi untuk mendapatkan yang terbaru
            log_files.sort(key=lambda x: os.path.getmtime(os.path.join(vis_data_path, x)))
            log_file = os.path.join(vis_data_path, log_files[-1])

    if not log_file or not os.path.exists(log_file):
        print(f"--- PERINGATAN: File log .json tidak ditemukan di dalam '{vis_data_path}'. Grafik stabilitas tidak dapat dibuat. ---")
        return

    print(f"--- INFO: Membaca log dari: {log_file} ---")
    
    # --- LOGIKA PARSING BARU DAN BENAR ---
    epoch_data = {}
    current_epoch = 0 # Variabel untuk melacak epoch terakhir yang terlihat

    with open(log_file, 'r') as f:
        for line in f:
            try:
                log = json.loads(line.strip())
                
                # KASUS 1: Ini adalah log training (karena memiliki 'loss' dan 'epoch')
                if 'loss' in log and 'epoch' in log:
                    epoch = log['epoch']
                    current_epoch = epoch # Perbarui epoch saat ini
                    
                    # Buat entri untuk epoch jika ini yang pertama kali
                    if epoch not in epoch_data:
                        epoch_data[epoch] = {'train_loss': [], 'val_map': 0}
                    
                    epoch_data[epoch]['train_loss'].append(log['loss'])

                # KASUS 2: Ini adalah log validasi (karena memiliki 'coco/bbox_mAP')
                elif 'coco/bbox_mAP' in log:
                    # Pastikan kita sudah melihat setidaknya satu epoch training
                    if current_epoch > 0 and current_epoch in epoch_data:
                        epoch_data[current_epoch]['val_map'] = log['coco/bbox_mAP']

            except (json.JSONDecodeError, KeyError):
                # Abaikan baris yang bukan JSON valid atau tidak memiliki kunci yang diharapkan
                continue

    # Bagian agregasi dan plotting tidak berubah dan sekarang akan berfungsi
    epochs, avg_losses, val_maps = [], [], []
    for epoch, data in sorted(epoch_data.items()):
        if data['train_loss'] and data['val_map'] > 0:
            epochs.append(epoch)
            avg_losses.append(np.mean(data['train_loss']))
            val_maps.append(data['val_map'])

    if not epochs:
        print("--- GAGAL: Tidak dapat mengekstrak pasangan data (loss & mAP) yang valid dari log. Periksa isi file log. ---")
        return

    # Sisa dari fungsi plotting...
    fig, ax1 = plt.subplots(figsize=(14, 7))
    color = 'tab:red'
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Rata-rata Training Loss', color=color)
    ax1.plot(epochs, avg_losses, 'o-', color=color, label='Training Loss (rata-rata)')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.grid(True, linestyle='--', alpha=0.6)

    ax2 = ax1.twinx()
    color = 'tab:blue'
    ax2.set_ylabel('Validation mAP', color=color)
    ax2.plot(epochs, val_maps, 's-', color=color, label='Validation mAP')
    ax2.tick_params(axis='y', labelcolor=color)

    plt.title('Kurva Stabilitas Training: Loss vs. Validation mAP')
    fig.tight_layout()
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='best')

    plot_path = os.path.join(output_dir, 'training_stability_curve.png')
    plt.savefig(plot_path, dpi=200)
    plt.close(fig)
    print(f">>> Grafik stabilitas training berhasil disimpan di: {plot_path}")


# ========== FUNGSI UNTUK CONFUSION MATRIX & PR CURVE ==========
def compute_iou_matrix(boxes1, boxes2):
    if boxes1.size == 0 or boxes2.size == 0: return np.zeros((boxes1.shape[0], boxes2.shape[0]), dtype=np.float32)
    x11, y11, x12, y12 = boxes1[:,0], boxes1[:,1], boxes1[:,2], boxes1[:,3]
    x21, y21, x22, y22 = boxes2[:,0], boxes2[:,1], boxes2[:,2], boxes2[:,3]
    inter_x1 = np.maximum(x11[:, None], x21[None, :]); inter_y1 = np.maximum(y11[:, None], y21[None, :])
    inter_x2 = np.minimum(x12[:, None], x22[None, :]); inter_y2 = np.minimum(y12[:, None], y22[None, :])
    inter_w = np.maximum(0, inter_x2 - inter_x1); inter_h = np.maximum(0, inter_y2 - inter_y1)
    inter = inter_w * inter_h
    area1 = (x12 - x11) * (y12 - y11); area2 = (x22 - x21) * (y22 - y21)
    union = area1[:, None] + area2[None, :] - inter
    return np.where(union > 0, inter / union, 0.0)

def greedy_match(iou_mat, iou_thr=0.5):
    matches = []; gt_used, pred_used = set(), set()
    if iou_mat.size == 0: return matches
    pairs = sorted([(i, j, iou_mat[i, j]) for i in range(iou_mat.shape[0]) for j in range(iou_mat.shape[1])], key=lambda x: x[2], reverse=True)
    for i, j, iou in pairs:
        if iou < iou_thr: break
        if i in gt_used or j in pred_used: continue
        gt_used.add(i); pred_used.add(j); matches.append((i, j))
    return matches
def evaluate_confusion_pr(model, dataset, output_dir, score_thr=0.3, iou_thr=0.5):
    print("\n>>> Memulai evaluasi kustom (Confusion Matrix, PR Curves & Metrik Per Kelas)...")
    base_dataset = dataset.dataset if hasattr(dataset, 'dataset') else dataset
    n_classes = len(CLASSES)
    conf_mat = np.zeros((n_classes, n_classes), dtype=np.int32)
    per_class_counts = {'TP': np.zeros(n_classes, dtype=np.int32), 'FP': np.zeros(n_classes, dtype=np.int32), 'FN': np.zeros(n_classes, dtype=np.int32)}
    pr_store = {c: {'scores': [], 'match': []} for c in range(n_classes)}

    for idx in range(len(base_dataset)):
        if idx % 50 == 0:
            print(f"  Mengevaluasi gambar {idx+1}/{len(base_dataset)}...")
        data_info = base_dataset.get_data_info(idx)
        img_path = data_info['img_path']
        gt_instances = data_info.get('instances', [])
        if gt_instances:
            gt_bboxes = np.array([inst['bbox'] for inst in gt_instances], dtype=np.float32)
            gt_labels = np.array([inst['bbox_label'] for inst in gt_instances], dtype=np.int64)
        else:
            gt_bboxes = np.empty((0, 4), dtype=np.float32)
            gt_labels = np.empty((0,), dtype=np.int64)
        
        result = inference_detector(model, img_path)
        pred = result.pred_instances
        keep = pred.scores >= score_thr
        pred_bboxes, pred_scores, pred_labels = pred.bboxes[keep].cpu().numpy(), pred.scores[keep].cpu().numpy(), pred.labels[keep].cpu().numpy()

        iou_mat = compute_iou_matrix(gt_bboxes, pred_bboxes)
        matches = greedy_match(iou_mat, iou_thr=iou_thr)
        matched_gt, matched_pred = {m[0] for m in matches}, {m[1] for m in matches}

        for gi, pj in matches:
            gt_c, pd_c = int(gt_labels[gi]), int(pred_labels[pj])
            if gt_c == pd_c:
                per_class_counts['TP'][pd_c] += 1
                pr_store[pd_c]['scores'].append(float(pred_scores[pj]))
                pr_store[pd_c]['match'].append(1)
            conf_mat[gt_c, pd_c] += 1

        for j, pd_c_int in enumerate(pred_labels.astype(int)):
            if j not in matched_pred:
                per_class_counts['FP'][pd_c_int] += 1
                pr_store[pd_c_int]['scores'].append(float(pred_scores[j]))
                pr_store[pd_c_int]['match'].append(0)

        for i, gt_c_int in enumerate(gt_labels.astype(int)):
            if i not in matched_gt:
                per_class_counts['FN'][gt_c_int] += 1

    # --- BAGIAN YANG DIKEMBALIKAN: Perhitungan dan Penulisan File per_class_metrics.txt ---
    eps = 1e-12
    prec = per_class_counts['TP'] / (per_class_counts['TP'] + per_class_counts['FP'] + eps)
    rec  = per_class_counts['TP'] / (per_class_counts['TP'] + per_class_counts['FN'] + eps)
    f1   = 2 * prec * rec / (prec + rec + eps)

    lines = ["KELAS,TP,FP,FN,Precision,Recall,F1"]
    for c, name in enumerate(CLASSES):
        lines.append(f"{name},{per_class_counts['TP'][c]},{per_class_counts['FP'][c]},{per_class_counts['FN'][c]},"
                     f"{prec[c]:.4f},{rec[c]:.4f},{f1[c]:.4f}")

    per_class_txt_path = os.path.join(output_dir, "per_class_metrics.txt")
    with open(per_class_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    print(f">>> Metrik per kelas disimpan di: {per_class_txt_path}")
    # --- AKHIR DARI BAGIAN YANG DIKEMBALIKAN ---

    # Plot Confusion Matrix
    fig_cm = plt.figure(figsize=(12, 10)); ax = fig_cm.add_subplot(111)
    im = ax.imshow(conf_mat, interpolation='nearest', cmap='viridis')
    ax.set_title(f"Confusion Matrix (IoU>{iou_thr}, Score>{score_thr})"); plt.colorbar(im, ax=ax)
    tick_marks = np.arange(len(CLASSES)); ax.set_xticks(tick_marks, labels=CLASSES, rotation=45, ha='right'); ax.set_yticks(tick_marks, labels=CLASSES)
    thresh = conf_mat.max() / 2.
    for i, j in itertools.product(range(conf_mat.shape[0]), range(conf_mat.shape[1])):
        ax.text(j, i, format(conf_mat[i, j], 'd'), ha="center", va="center", color="white" if conf_mat[i, j] > thresh else "black")
    plt.ylabel('True Label'); plt.xlabel('Predicted Label'); plt.tight_layout()
    cm_path = os.path.join(output_dir, "confusion_matrix.png"); fig_cm.savefig(cm_path, dpi=200); plt.close(fig_cm)
    print(f">>> Confusion matrix disimpan di: {cm_path}")

    # Plot PR Curves
    for c in range(len(CLASSES)):
        # ... (sisa kode plotting PR curve tetap sama)
        scores, match = np.array(pr_store[c]['scores']), np.array(pr_store[c]['match'])
        if scores.size == 0: continue
        order = np.argsort(-scores); scores, match = scores[order], match[order]
        tp_cum, fp_cum = np.cumsum(match), np.cumsum(1 - match)
        precision = tp_cum / np.maximum(tp_cum + fp_cum, 1)
        total_pos = per_class_counts['TP'][c] + per_class_counts['FN'][c]
        recall = tp_cum / max(total_pos, 1) if total_pos > 0 else np.zeros_like(tp_cum)
        fig_pr = plt.figure(); plt.plot(recall, precision, '-o', markersize=4); plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR Curve - {CLASSES[c]}"); plt.grid(); plt.xlim(-0.05, 1.05); plt.ylim(-0.05, 1.05)
        pr_path = os.path.join(output_dir, f"pr_curve_{c:02d}_{CLASSES[c].replace(' ','_')}.png"); fig_pr.savefig(pr_path, dpi=200, bbox_inches='tight'); plt.close(fig_pr)
    print(f">>> PR curves disimpan di: {output_dir}")

create_stability_curve_by_epoch(runner.work_dir, report_dir)

# --- B. Alur Eksekusi untuk Evaluasi Kustom ---
# Path ke checkpoint sekarang juga didasarkan pada runner.work_dir
best_checkpoint_file = None
best_ckpts = [f for f in os.listdir(runner.work_dir) if f.startswith('best_') and f.endswith('.pth')]
if best_ckpts:
    best_ckpts.sort()
    best_checkpoint_file = os.path.join(runner.work_dir, best_ckpts[-1])

if best_checkpoint_file and os.path.exists(best_checkpoint_file):
    print(f"\n--- INFO: Memuat checkpoint terbaik untuk evaluasi kustom: {best_checkpoint_file} ---")
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = init_detector(cfg, best_checkpoint_file, device=device)
    
    # Jalankan evaluasi kustom menggunakan fungsi yang sudah diperbarui
    evaluate_confusion_pr(model, val_dataset, report_dir, score_thr=0.3, iou_thr=0.5)
else:
    print(f"--- PERINGATAN: Checkpoint terbaik tidak ditemukan di '{runner.work_dir}'. Evaluasi kustom dilewati. ---")

print("\n--- Semua proses (training dan analisis) telah selesai. ---")


--- Memulai Analisis Pasca-Training. Laporan akan disimpan di: ./outputs_model_dengan_grafik\reports_epoch30_batchsize8_lr0.0001(2) ---

>>> Menjalankan evaluasi COCO standar...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
10/29 11:32:11 - mmengine - INFO - Epoch(test) [10/61]    eta: 0:01:39  time: 1.9507  data_time: 1.7269  memory: 1152  
10/29 11:32:13 - mmengine - INFO - Epoch(test) [20/61]    eta: 0:00:44  time: 1.0812  data_time: 0.8652  memory: 1152  
10/29 11:32:15 - mmengine - INFO - Epoch(test) [30/61]    eta: 0:00:24  time: 0.7912  data_time: 0.5781  memory: 1152  
10/29 11:32:17 - mmengine - INFO - Epoch(test) [40/61]    eta: 0:00:13  time: 0.6463  data_time: 0.4345  memory: 1152  
10/29 11:32:19 - mmengine - INFO - Epoch(test) [50/61]    eta: 0:00:06  time: 0.5629  data_time: 0.3485  memory: 1152  
10/29 11:32:21 - mmengine - INFO - Epoch(test) [60/61] 

In [12]:

# ========== STEP 11: INFERENCE + RINGKASAN JUMLAH PER JENIS ==========

# --- Setup Direktori dan Visualizer ---
vis_save_dir = os.path.join(cfg.work_dir, 'inference_results')
os.makedirs(vis_save_dir, exist_ok=True)

visualizer = DetLocalVisualizer(
    vis_backends=[dict(type='LocalVisBackend')],
    name='visualizer',
    save_dir=vis_save_dir
)

# ========== PERBAIKAN UTAMA DI SINI ==========
# Ambil metainfo langsung dari model yang sudah diinisialisasi.
# Ini adalah cara yang paling aman dan direkomendasikan.
visualizer.dataset_meta = model.dataset_meta
# ============================================

# --- Proses Inferensi pada Gambar Uji ---
# Ganti dengan path gambar yang ingin Anda uji
img_path = 'dataset/10_jenis_ikan/test/Discuss_149_jpg.rf.7eae5bed233fd271fbdf51d38365de39.jpg'

if not os.path.exists(img_path):
    print(f"ERROR: Gambar uji tidak ditemukan di '{img_path}'. Melewati langkah inferensi.")
else:
    print(f">>> Melakukan inferensi pada gambar: {os.path.basename(img_path)}")
    result = inference_detector(model, img_path)
    pred_instances = result.pred_instances
    scores = pred_instances.scores.cpu().numpy()
    labels = pred_instances.labels.cpu().numpy()

    # Tentukan score threshold untuk memfilter deteksi
    score_thr = 0.3
    keep_indices = scores >= score_thr
    
    # Hitung jumlah ikan yang terdeteksi per kelas
    labels_kept = labels[keep_indices]
    class_names = visualizer.dataset_meta['classes'] # Gunakan kelas dari visualizer
    counts = {name: int((labels_kept == i).sum()) for i, name in enumerate(class_names)}
    total_detected = int(keep_indices.sum())

    # --- Tampilkan dan Simpan Ringkasan Deteksi ---
    print("\n>>> Ringkasan Deteksi (Score Threshold > " + str(score_thr) + "):")
    for class_name, count in counts.items():
        if count > 0:
            print(f"- {class_name}: {count}")
    print("-" * 20)
    print(f"Total Ikan Terdeteksi: {total_detected}")

    # Simpan ringkasan ke file teks
    summary_txt_path = os.path.join(vis_save_dir, "detection_summary.txt")
    with open(summary_txt_path, "w", encoding="utf-8") as f:
        f.write(f"Hasil Deteksi pada: {os.path.basename(img_path)}\n")
        f.write(f"Score Threshold: {score_thr}\n")
        f.write("="*30 + "\n")
        for class_name, count in counts.items():
             if count > 0:
                f.write(f"{class_name}: {count}\n")
        f.write("="*30 + "\n")
        f.write(f"TOTAL TERDETEKSI: {total_detected}\n")
    print(f">>> Ringkasan deteksi disimpan di: {summary_txt_path}")

    # --- Visualisasi Hasil Deteksi dan Simpan ke File ---
    img = mmcv.imread(img_path)
    img = mmcv.imconvert(img, 'bgr', 'rgb') # Konversi BGR (OpenCV) ke RGB (Matplotlib)
    
    output_image_path = os.path.join(vis_save_dir, f"result_{os.path.basename(img_path)}")
    
    visualizer.add_datasample(
        name='prediction',
        image=img,
        data_sample=result,
        draw_gt=False,
        show=False,           # Set True jika Anda ingin gambar muncul di jendela popup
        wait_time=0,
        pred_score_thr=score_thr,
        out_file=output_image_path
    )
    print(f">>> Gambar hasil deteksi disimpan di: {output_image_path}")

>>> Melakukan inferensi pada gambar: Discuss_149_jpg.rf.7eae5bed233fd271fbdf51d38365de39.jpg

>>> Ringkasan Deteksi (Score Threshold > 0.3):
- Gurami: 1
--------------------
Total Ikan Terdeteksi: 1
>>> Ringkasan deteksi disimpan di: ./outputs_model_dengan_grafik\inference_results\detection_summary.txt
>>> Gambar hasil deteksi disimpan di: ./outputs_model_dengan_grafik\inference_results\result_Discuss_149_jpg.rf.7eae5bed233fd271fbdf51d38365de39.jpg
